In [ ]:
import numpy as np
from PIL import Image

# Load the ZNP file
data = np.load("/content/octmnist.npz")

# Access images and labels
images = data["images"]
labels = data["labels"]

# View the first image
image = np.squeeze(images[0]) * 255.0
image = Image.fromarray(image.astype(np.uint8)).convert("L")
image.show()

# Print the corresponding label
print(f"Label: {labels[0]}")

In [ ]:
#dataset code
import os
import numpy as np
from PIL import Image
from torch.utils.data import Dataset
from medmnist.info import INFO, HOMEPAGE, DEFAULT_ROOT


class MedMNIST(Dataset):

    flag = ...

    def __init__(self,
                 split,
                 transform=None,
                 target_transform=None,
                 download=False,
                 as_rgb=False,
                 root=DEFAULT_ROOT):
        ''' dataset
        :param split: 'train', 'val' or 'test', select subset
        :param transform: data transformation
        :param target_transform: target transformation

        '''

        self.info = INFO[self.flag]

        if root is not None and os.path.exists(root):
            self.root = root
        else:
            raise RuntimeError("Failed to setup the default `root` directory. " +
                               "Please specify and create the `root` directory manually.")

        if download:
            self.download()

        if not os.path.exists(
                os.path.join(self.root, "{}.npz".format(self.flag))):
            raise RuntimeError('Dataset not found. ' +
                               ' You can set `download=True` to download it')

        npz_file = np.load(os.path.join(self.root, "{}.npz".format(self.flag)))

        self.split = split
        self.transform = transform
        self.target_transform = target_transform
        self.as_rgb = as_rgb

        if self.split == 'train':
            self.imgs = npz_file['train_images']
            self.labels = npz_file['train_labels']
        elif self.split == 'val':
            self.imgs = npz_file['val_images']
            self.labels = npz_file['val_labels']
        elif self.split == 'test':
            self.imgs = npz_file['test_images']
            self.labels = npz_file['test_labels']
        else:
            raise ValueError

    def __len__(self):
        return self.imgs.shape[0]

    def __repr__(self):
        '''Adapted from torchvision.ss'''
        _repr_indent = 4
        head = f"Dataset {self.__class__.__name__} ({self.flag})"
        body = [f"Number of datapoints: {self.__len__()}"]
        body.append(f"Root location: {self.root}")
        body.append(f"Split: {self.split}")
        body.append(f"Task: {self.info['task']}")
        body.append(f"Number of channels: {self.info['n_channels']}")
        body.append(f"Meaning of labels: {self.info['label']}")
        body.append(f"Number of samples: {self.info['n_samples']}")
        body.append(f"Description: {self.info['description']}")
        body.append(f"License: {self.info['license']}")

        lines = [head] + [" " * _repr_indent + line for line in body]
        return '\n'.join(lines)

    def download(self):
        try:
            from torchvision.datasets.utils import download_url
            download_url(url=self.info["url"],
                         root=self.root,
                         filename="{}.npz".format(self.flag),
                         md5=self.info["MD5"])
        except:
            raise RuntimeError('Something went wrong when downloading! ' +
                               'Go to the homepage to download manually. ' +
                               HOMEPAGE)


class MedMNIST2D(MedMNIST):

    def __getitem__(self, index):
        '''
        return: (without transform/target_transofrm)
            img: PIL.Image
            target: np.array of `L` (L=1 for single-label)
        '''
        img, target = self.imgs[index], self.labels[index].astype(int)
        img = Image.fromarray(img)

        if self.as_rgb:
            img = img.convert('RGB')

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def save(self, folder, postfix="png", write_csv=True):

        from medmnist.utils import save2d

        save2d(imgs=self.imgs,
               labels=self.labels,
               img_folder=os.path.join(folder, self.flag),
               split=self.split,
               postfix=postfix,
               csv_path=os.path.join(folder, f"{self.flag}.csv") if write_csv else None)

    def montage(self, length=20, replace=False, save_folder=None):
        from medmnist.utils import montage2d

        n_sel = length * length
        sel = np.random.choice(self.__len__(), size=n_sel, replace=replace)

        montage_img = montage2d(imgs=self.imgs,
                                n_channels=self.info['n_channels'],
                                sel=sel)

        if save_folder is not None:
            if not os.path.exists(save_folder):
                os.makedirs(save_folder)
            montage_img.save(os.path.join(save_folder,
                                          f"{self.flag}_{self.split}_montage.jpg"))

        return montage_img


class MedMNIST3D(MedMNIST):

    def __getitem__(self, index):
        '''
        return: (without transform/target_transofrm)
            img: an array of 1x28x28x28 or 3x28x28x28 (if `as_RGB=True`), in [0,1]
            target: np.array of `L` (L=1 for single-label)
        '''
        img, target = self.imgs[index], self.labels[index].astype(int)

        img = np.stack([img/255.]*(3 if self.as_rgb else 1), axis=0)

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def save(self, folder, postfix="gif", write_csv=True):
        from medmnist.utils import save3d

        assert postfix == "gif"

        save3d(imgs=self.imgs,
               labels=self.labels,
               img_folder=os.path.join(folder, self.flag),
               split=self.split,
               postfix=postfix,
               csv_path=os.path.join(folder, f"{self.flag}.csv") if write_csv else None)

    def montage(self, length=20, replace=False, save_folder=None):
        assert self.info['n_channels'] == 1

        from medmnist.utils import montage3d, save_frames_as_gif
        n_sel = length * length
        sel = np.random.choice(self.__len__(), size=n_sel, replace=replace)

        montage_frames = montage3d(imgs=self.imgs,
                                   n_channels=self.info['n_channels'],
                                   sel=sel)

        if save_folder is not None:
            if not os.path.exists(save_folder):
                os.makedirs(save_folder)

            save_frames_as_gif(montage_frames,
                               os.path.join(save_folder,
                                            f"{self.flag}_{self.split}_montage.gif"))

        return montage_frames


class PathMNIST(MedMNIST2D):
    flag = "pathmnist"


class OCTMNIST(MedMNIST2D):
    flag = "octmnist"


class PneumoniaMNIST(MedMNIST2D):
    flag = "pneumoniamnist"


class ChestMNIST(MedMNIST2D):
    flag = "chestmnist"


class DermaMNIST(MedMNIST2D):
    flag = "dermamnist"


class RetinaMNIST(MedMNIST2D):
    flag = "retinamnist"


class BreastMNIST(MedMNIST2D):
    flag = "breastmnist"


class BloodMNIST(MedMNIST2D):
    flag = "bloodmnist"


class TissueMNIST(MedMNIST2D):
    flag = "tissuemnist"


class OrganAMNIST(MedMNIST2D):
    flag = "organamnist"


class OrganCMNIST(MedMNIST2D):
    flag = "organcmnist"


class OrganSMNIST(MedMNIST2D):
    flag = "organsmnist"


class OrganMNIST3D(MedMNIST3D):
    flag = "organmnist3d"


class NoduleMNIST3D(MedMNIST3D):
    flag = "nodulemnist3d"


class AdrenalMNIST3D(MedMNIST3D):
    flag = "adrenalmnist3d"


class FractureMNIST3D(MedMNIST3D):
    flag = "fracturemnist3d"


class VesselMNIST3D(MedMNIST3D):
    flag = "vesselmnist3d"


class SynapseMNIST3D(MedMNIST3D):
    flag = "synapsemnist3d"


# backward-compatible
OrganMNISTAxial = OrganAMNIST
OrganMNISTCoronal = OrganCMNIST
OrganMNISTSagittal = OrganSMNIST

In [ ]:


from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from sklearn.model_selection import train_test_split
import lime
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16, ResNet50
from scipy.interpolate import interp2d

# Define custom CNN loss function
def custom_loss(y_true, y_pred):
    confidence_loss = tf.keras.losses.MeanSquaredError(reduction="auto")(y_true[0], y_pred[0])
    localization_loss = tf.keras.losses.MeanSquaredError(reduction="auto")(y_true[1], y_pred[1])
    class_loss = tf.keras.losses.BinaryCrossentropy(reduction="auto")(y_true[2], y_pred[2])

    combined_loss = 0.5 * confidence_loss + 0.3 * localization_loss + 0.2 * class_loss

    return combined_loss

class CNN(models.Model):
    def __init__(self, num_classes, input_shape=(28, 28, 1)):
        super(CNN, self).__init__()
        self.num_classes = num_classes
        self.input_shape = input_shape

        self.vgg16 = VGG16(weights="imagenet", include_top=False, input_shape=input_shape)
        self.resnet50 = ResNet50(weights="imagenet", include_top=False, input_shape=input_shape)

        for layer in self.vgg16.layers:
            layer.trainable = False
        for layer in self.resnet50.layers:
            layer.trainable = False

        inputs = layers.Input(shape=input_shape)
        self.vgg_features = self.vgg16(inputs)
        self.resnet_features = self.resnet50(inputs)

        merged_features = layers.concatenate([self.vgg_features, self.resnet_features])

        self.conv1 = layers.Conv2D(32, kernel_size=(3, 3), activation="relu")(merged_features)
        self.maxpool1 = layers.MaxPooling2D(pool_size=(2, 2))(self.conv1)
        self.conv2 = layers.Conv2D(64, kernel_size=(3, 3), activation="relu")(self.maxpool1)
        self.maxpool2 = layers.MaxPooling2D(pool_size=(2, 2))(self.conv2)
        # ... (additional layers)

        self.flatten = layers.Flatten()(self.maxpool2)
        self.fc1 = layers.Dense(128, activation="relu")(self.flatten)
        self.fc2 = layers.Dense(64, activation="relu")(self.fc1)
        # ... (additional layers)

        self.fc3 = layers.Dense(num_classes, activation="softmax")(self.fc2)

    def call(self, inputs):
        return self.fc3(self.vgg16(inputs) + self.resnet50(inputs))

def resize(image, desired_size=(256, 256)):
    scale_y = desired_size[0] / image.shape[0]
    scale_x = desired_size[1] / image.shape[1]

    rows = np.arange(0, image.shape[0])
    cols = np.arange(0, image.shape[1])

    rows_resized = np.arange(0, desired_size[0], scale_y)
    cols_resized = np.arange(0, desired_size[1], scale_x)

    interpolator_r = interp2d(cols, rows, image[:, :, 0], kind='cubic', bounds_error=False, fill_value=0)
    interpolator_g = interp2d(cols, rows, image[:, :, 1], kind='cubic', bounds_error=False, fill_value=0)
    interpolator_b = interp2d(cols, rows, image[:, :, 2], kind='cubic', bounds_error=False, fill_value=0)

    resized_image_r = interpolator_r(cols_resized, rows_resized)
    resized_image_g = interpolator_g(cols_resized, rows_resized)
    resized_image_b = interpolator_b(cols_resized, rows_resized)

    resized_image = np.stack([resized_image_r, resized_image_g, resized_image_b], axis=-1)

    return resized_image

def preprocess_cnv(image):
    original_shape = image.shape
    filtered_image = image + np.random.normal(size=original_shape, scale=0.05)
    enhanced_image = filtered_image + np.mean(filtered_image) - np.mean(image)
    roi_size = int(min(original_shape[0], original_shape[1]) / 2)
    roi_x = int((original_shape[0] - roi_size) / 2)
    roi_y = int((original_shape[1] - roi_size) / 2)
    roi_image = enhanced_image[roi_x:roi_x + roi_size, roi_y:roi_y + roi_size]
    return roi_image

def preprocess_normal(image, dtype=np.float32):
    image = image.astype(dtype)
    image = (image - np.min(image)) / (np.max(image) - np.min(image))
    return image

def load_data(path="/content/octmnist.npz"):
    with np.load(path) as data:
        images = data["images"]
        labels = data["labels"]

    labels = np.array(labels)

    return images, labels

X, y = load_data()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=5,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
)

model = CNN(num_classes=2)
model.compile(loss=custom_loss, optimizer="adam", metrics=["accuracy"])

early_stopping = callbacks.EarlyStopping(monitor="val_accuracy", patience=5)

model.fit(datagen.flow(X_train, y_train, batch_size=32),
          epochs=20,
          validation_data=(X_val, y_val),
          callbacks=[early_stopping])

loss, accuracy = model.evaluate(X_val, y_val)
print(f"\nModel Evaluation:\nLoss: {loss:.4f}\nAccuracy: {accuracy:.4f}")

# Make predictions on the validation set
y_pred = model.predict(X_val)
y_pred_class = np.argmax(y_pred, axis=1)

# Convert one-hot encoded labels to class labels
y_val_class = np.argmax(y_val, axis=1)

# Confusion Matrix
conf_matrix = confusion_matrix(y_val_class, y_pred_class)
print("\nConfusion Matrix:")
print(conf_matrix)

# Precision
precision = precision_score(y_val_class, y_pred_class)
print("Precision:", precision)

# Recall
recall = recall_score(y_val_class, y_pred_class)
print("Recall:", recall)

# F1 Score
f1 = f1_score(y_val_class, y_pred_class)
print("F1 Score:", f1)

def predict(image_path):
    image = Image.open(image_path).convert("L")
    image = np.array(image, dtype=np.float32) / 255.0
    image = preprocess_cnv(image)
    image = resize(image, (28, 28))
    image = np.expand_dims(image, axis=0)
    prediction = model.predict(image)[0]
    return prediction

def transfer_style(image_path, target_style_path, desired_size=256):
    original_image = Image.open(image_path).convert("RGB")
    target_image = Image.open(target_style_path).convert("RGB")
    original_image = np.array(original_image, dtype=np.float32) / 255.0
    target_image = np.array(target_image, dtype=np.float32) / 255.0
    original_image = resize(original_image, (desired_size, desired_size))
    target_image = resize(target_image, (desired_size, desired_size))
    cyclegan = tf.keras.models.load_model("path/to/cyclegan_model.h5")
    transferred_image = cyclegan.predict([original_image, target_image])[0]
    visualize_lime(original_image, transferred_image)
    return transferred_image

def visualize_lime(original_image, transferred_image):
    explainer = lime.lime_image.LimeImageExplainer()
    explanation = explainer.explain_instance(transferred_image, model.predict, top_labels=5)
    heatmap, mask = explanation.get_image_and_mask(original_image, num_features=5)
    plt.imshow(original_image)
    plt.imshow(heatmap, alpha=0.5)
    plt.show()

# Example usage
image_path_cnv = "path/to/cnv_image.png"
prediction_cnv = predict(image_path_cnv)
print("CNV Prediction:", prediction_cnv)

image_path_style = "path/to/style_transfer_image.png"
target_style_path = "path/to/style_image.png"
transferred_image = transfer_style(image_path_style, target_style_path)


ModuleNotFoundError: ignored